In [23]:
from astropy.table import Table
import numpy as np
import pandas as pd

In [105]:
lcm = Table.read('Results/school_lcm_mg.tex').to_pandas().drop(columns=['col0'])
lin_prog = Table.read('Results/school_linear_prog_mg.tex').to_pandas().drop(columns=['col0'])
np_prog = Table.read('Results/school_ensemble_prog_mg.tex').to_pandas().drop(columns=['col0'])

In [106]:
def clean_table(table, method_name, include_query=False):
    table['T'] = table['T'].fillna(0).astype(int)
    for c in table.columns:
        if np.array_equal(table.loc[:, c], table.loc[:, c].astype(int)):
            table[c] = table[c].astype(int)
    table = table[['T', 'S3', 'X1', 'C1', 'C2']]
    table['C2'] -= 1
    table['X1'] = (table['X1']*100).astype(int)
    table['S3'] =  table['S3'].astype(int)
#     table.iloc[:-1, -2:] = round(table.iloc[:-1, -2:], 2)
#     table.iloc[-1, -3:] = round(table.iloc[-1, -3:], 4)
    table = table.astype(str)
    table = table.iloc[:-1, :]
    table = table.T
    if include_query:
        query = table.iloc[:, [0]]
        query.columns = ['Query unit']
    table = table.iloc[:, 1:].T
    table = table[table['T'] == '0']    
    table = table.T
    table.columns = [method_name] + ['' for i in range(table.shape[1]-1)]
    if include_query:
        return table, query
    return table

In [107]:
lcm, query = clean_table(lcm, 'LCM', include_query=True)
lpgm = clean_table(lin_prog, 'Linear PGM')
npgm = clean_table(np_prog, 'Nonparametric PGM')

In [108]:
total = query.join(lcm, lsuffix='').join(lpgm, lsuffix='_').join(npgm, lsuffix='__')
total.columns = [c.replace('_', '') for c in total.columns]

In [109]:
total.T

,T,S3,X1,C1,C2
Query unit,1,5,-5,5,1
LCM,0,5,-5,5,1
,0,5,-5,5,1
,0,5,-5,10,1
,0,5,-5,2,1
,0,5,-5,2,1
,0,5,-5,2,1
,0,5,-5,2,1
,0,5,-5,2,1
,0,5,-24,5,1


In [110]:
total.T.to_latex('test.tex')

S3
True
C2
True
XC
True
X1
False
C1
True
X3
False
T
True
